In [1]:
import sys

from recommendation.ratings_matrix import create_ratings_matrix
sys.path.append('/Users/garethmiskimmin/streamline-app/api')

ratings_matrix = create_ratings_matrix()

Function 'get_all_user_interactions' executed in 0.15 seconds.
Function '__compute_first_interaction_score' executed in 0.15 seconds.
Function '__add_time_decay' executed in 0.26 seconds.
Function '__normalize_scores' executed in 0.01 seconds.
Function 'create_ratings_matrix' executed in 17.19 seconds.


In [3]:
from recommendation.content_based_filtering_service import get_content_based_filtering_model

cbf_model = get_content_based_filtering_model(ratings_matrix)

Function 'get_movies_metadata' executed in 54.51 seconds.
Function '__build_metadata_vectorized' executed in 1.59 seconds.


Building user profiles:   0%|          | 0/63 [00:00<?, ?it/s]

Function '__tf_idf_on_metadata' executed in 8.73 seconds.


Function 'get_content_based_filtering_model' executed in 89.13 seconds.


In [10]:
import pandas as pd
import psycopg
from psycopg.rows import dict_row
from common.utils.utils import DB_CONFIG


query = """
SELECT 
    user_id, 
    movie_id, 
    predicted_score
FROM user_recommendations
WHERE user_id = ANY(%s)
"""
user_ids = [9,10]

with psycopg.connect(**DB_CONFIG) as conn:
    with conn.cursor(row_factory=dict_row) as cur:
        # ✅ Fetch onboarding movies
        cur.execute(query, [user_ids])

        user_recommendations = cur.fetchall()

combined_frame = pd.DataFrame(user_recommendations)

merged_frame = combined_frame.groupby(["movie_id"])["predicted_score"].mean()

top = merged_frame.sort_values(ascending=False).head(100)